## Stage 1: Preprocessing & Cleaning

### Setup & Data Loading

In [176]:
# Imports
import pandas as pd
import re
import os
import ast
import string
import nltk
import spacy
import requests
import contractions
from langdetect import detect 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from unidecode import unidecode
from bs4 import BeautifulSoup
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tim/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/tim/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tim/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /Users/tim/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [177]:
# Download a spacy model, can also be adjusted (medium = en_core_web_sm, large = en_core_web_lg)
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 7.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [178]:
# Load the data
raw_data = pd.read_csv('../data/esg_documents_for_dax_companies.csv', delimiter = '|', index_col = 0)

In [179]:
# Check loaded data and reset index
raw_data = raw_data.reset_index(drop=True)
raw_data.head(10)

,company,content,datatype,date,domain,esg_topics,internal,symbol,title,url
0,Beiersdorf AG,Sustainability Highlight Report CARE BEYOND SK...,sustainability_report,2021-03-31,NaN,"['CleanWater', 'GHGEmission', 'ProductLiabilit...",1,BEI,BeiersdorfAG Sustainability Report 2021,NaN
1,Deutsche Telekom AG,Corporate Responsibility Report 2021 2 Content...,sustainability_report,2021-03-31,NaN,"['DataSecurity', 'Iso50001', 'GlobalWarming', ...",1,DTE,DeutscheTelekomAG Sustainability Report 2021,NaN
2,Vonovia SE,VONOVIA SE SUSTAINABILITY REPORT 2021 =For a S...,sustainability_report,2021-03-31,NaN,"['Whistleblowing', 'DataSecurity', 'Vaccine', ...",1,VNA,VonoviaSE Sustainability Report 2021,NaN
3,Merck KGaA,Sustainability Report 2021 TABLE OF CONTENTS S...,sustainability_report,2021-03-31,NaN,"['DataSecurity', 'DataMisuse', 'DrugResistance...",1,MRK,MerckKGaA Sustainability Report 2021,NaN
4,MTU,Our ideas and concepts FOR A SUSTAINABLE FUTUR...,sustainability_report,2020-03-31,NaN,"['WorkLifeBalance', 'Corruption', 'AirQuality'...",1,MTX,MTUAeroEngines Sustainability Report 2020,NaN
5,E ONSE,#StandWithUkraine Sustainability Report 2021 C...,sustainability_report,2021-03-31,NaN,"['DataSecurity', 'Iso50001', 'GlobalWarming', ...",1,EOAN,E.ONSE Sustainability Report 2021,NaN
6,RWE AG,Focus on tomorrow. Sustainability Report 2021 ...,sustainability_report,2021-03-31,NaN,"['WorkLifeBalance', 'Corruption', 'Iso50001', ...",1,RWE,RWEAG Sustainability Report 2021,NaN
7,Heidelberg Cement AG,Annual Report 2021 HeidelbergCement at a glanc...,annual_report,2021-03-31,NaN,"['WorkLifeBalance', 'Vaccine', 'DataSecurity',...",1,HEI,HeidelbergCementAG Annual Report 2021,NaN
8,Heidelberg Cement AG,Company Strategy & Business & Product & Produc...,sustainability_report,2020-03-31,NaN,"['CleanWater', 'Corruption', 'Whistleblowing',...",1,HEI,HeidelbergCementAG Sustainability Report 2020,NaN
9,Siemens AG,Sustainability 1 Siemens 2 Our 3 Governance – ...,sustainability_report,2020-03-31,NaN,"['DataSecurity', 'Iso50001', 'EmployeeTurnover...",1,SIE,SiemensAG Sustainability Report 2020,NaN


**Column descriptions**
- symbol: stock symbol of the company
- company: company name
- date: publication date of document
- title: document title
- content: document content
- datatype: document type
- internal: is this a report by company (1) or a third-party document (0)
- domain (optional): Web domain where the document was published
- url (optional): URL where the document can be accessed
- esg_topics (optional): ESG topics extracted from the data using our internal NLP

In [180]:
# Check shape (row and column amount)
raw_data.shape

(11188, 10)

In [181]:
# Check datatypes
raw_data.dtypes

company       object
content       object
datatype      object
date          object
domain        object
esg_topics    object
internal       int64
symbol        object
title         object
url           object
dtype: object

The data is loaded correctly.

In [182]:
# Small checkpoint function to save intermediary processing steps and enhance development
def csv_checkpoint(df, filename='checkpoint'):

    if not os.path.exists('../data/checkpoints/'):
        os.makedirs('../data/checkpoints/')

    # Save DataFrame to CSV
    df.to_csv(f'../data/checkpoints/{filename}.csv', index=False, sep = '|')
    print(f'Saved DataFrame to {filename}.csv')
    
    # Load CSV back into DataFrame
    df = pd.read_csv(f'../data/checkpoints/{filename}.csv', delimiter = '|')
    print(f'Loaded DataFrame from {filename}.csv')
    
    return df

## General Data Cleaning

As initial data cleaning steps, the following is conducted:
- Rows with missing "content" were dropped to prevent any missing data-related issues. Missing data can create gaps in the data and lead to errors or distortions in the analysis.
- The "URL" column was removed as the relevant information was available in the "domain" column. Removing redundant columns simplifies the data set and makes it easier to work with
- Duplicate entries were identified and removed, resulting in a cleaner and more concise dataset. Duplicates can distort the data and lead to biased analysis. 
- Language checking was conducted and all rows with non-English content were dropped to ensure consistent language. Language inconsistencies can create bias in the data and lead to inaccurate conclusions. Therefore, it is important to ensure that the data is consistent in language to prevent linguistic biases.
- "Date" is formatted as a date and wrong dates, e.g. "bayer-03-31" are replaced with a default date (2023-03-31).
- Remove company name parts like "AG" for clarity
- The "sample" method was used to check the data for representativeness and potential issues.

In [183]:
general_cleaned_data = raw_data.copy(deep=True)

In [184]:
# Delete all rows with no content, e.g. no report
general_cleaned_data = general_cleaned_data.dropna(subset=['content'])

In [185]:
# Drop the "url" column, since the most relevant information from an analysis perspective is already in the "domain" column (e.g. the source of the report)
general_cleaned_data = general_cleaned_data.drop(columns=['url'])

In [186]:
# Check for duplicates and delete them
duplicates = general_cleaned_data[general_cleaned_data.duplicated()]
print(f'Duplicated rows: {len(duplicates)}')
general_cleaned_data = general_cleaned_data.drop_duplicates()

Duplicated rows: 6


In [187]:
# Check for other languange than English
general_cleaned_data['language'] = general_cleaned_data['content'].apply(lambda x: detect(x))
not_english = len(general_cleaned_data) - len(general_cleaned_data.loc[general_cleaned_data['language'] == 'en'])

# Drop rows with other languange, since other languanges influences to quality of the later analysis
general_cleaned_data = general_cleaned_data.loc[general_cleaned_data['language'] == 'en']

print(f'Deleted amount of rows with language other ehan English: {not_english}')
general_cleaned_data.drop(['language'], axis=1, inplace=True)

Deleted amount of rows with language other ehan English: 106


In [188]:
# Correct the dates to ISO standard
def find_incorrect_dates(data):
    incorrect_dates = []

    for index, row in data.iterrows():
        try:
            pd.to_datetime(row['date'], format='%Y-%m-%d', errors='raise')
        except ValueError:
            incorrect_dates.append((index, row['date']))

    return incorrect_dates

incorrect_date_rows = find_incorrect_dates(general_cleaned_data)
print("Incorrectly formatted dates:")
for index, date in incorrect_date_rows:
    print(f"Row {index}: {date}")

Incorrectly formatted dates:
Row 13: p.DE-03-31
Row 18: p.DE-03-31
Row 20: bayer-03-31
Row 22: p.DE-03-31
Row 25: p.DE-03-31
Row 26: p.DE-03-31
Row 31: p.DE-03-31
Row 32: p.DE-03-31
Row 33: p.DE-03-31
Row 37: p.DE-03-31
Row 41: p.DE-03-31
Row 50: p.DE-03-31
Row 78: p.DE-03-31
Row 80: p.DE-03-31
Row 86: p.DE-03-31
Row 87: p.DE-03-31
Row 88: p.DE-03-31


In [189]:
# Correct the wrong formatted dates and set default date
def correct_date_format(data):
    data['date'] = pd.to_datetime(data['date'], errors='coerce').fillna('2022-03-31')
    return data

general_cleaned_data = correct_date_format(general_cleaned_data)

In [190]:
# Replace company name parts like "AG" to have a cleaner name
general_cleaned_data['company'] = general_cleaned_data['company'].str.replace(' AG', '')
general_cleaned_data['company'] = general_cleaned_data['company'].str.replace(' SE', '')
general_cleaned_data['company'] = general_cleaned_data['company'].str.replace(' KGaA', '')

In [191]:
# Drop rows with no content, e.g. no report
general_cleaned_data = general_cleaned_data.dropna(subset=['content'])

In [192]:
# Check the date with some samples
general_cleaned_data.sample(5)

,company,content,datatype,date,domain,esg_topics,internal,symbol,title
9861,Siemens Healthineers,"October 23, 2022 — Meeting with a medical phys...",general,2022-10-23,itnonline,['Social'],0,SHL,Medical Physicist Consults with Patients can H...
7166,Porsche,"Vuzix Corporation, a leading supplier of Smart...",tech,2021-05-20,aithority,"['Mentoring', 'Environment']",0,PAH3,"Vuzix, Softfoundry and Porsche China to Host i..."
203,Adidas,"ADVERTISEMENT On Tuesday morning, adidas anno...",general,2022-10-25,sneakernews,"['Antisemitism', 'Diversity', 'HateSpeech']",0,ADS,"adidas Breaks Up With Kanye West, Keeps Yeezy ..."
3635,Daimler,A forensic scientist has given evidence at the...,general,2022-02-10,irishexaminer,"['Racism', 'HumanRights']",0,DAI,Scientist gives evidence about drugs found aft...
7845,RWE,"Hi, what are you looking for? By Published Tho...",tech,2021-03-29,digitaljournal,"['ClimateChange', 'CarbonDioxide', 'Coal', 'Fo...",0,RWE,'Hambi stays! ': Thousands join German forest ...


In [193]:
# Change name of "Muenchener Rueckversicherungs Gesellschaft AGin Muenchen" to something more readable
general_cleaned_data['company'] = general_cleaned_data['company'].replace('Muenchener Rueckversicherungs Gesellschaftin Muenchen', 'Munich R')

In [194]:
# Create checkpoint file
general_cleaned_data = csv_checkpoint(general_cleaned_data, 'general_cleaned_data')

Saved DataFrame to general_cleaned_data.csv
Loaded DataFrame from general_cleaned_data.csv


## Text Data Cleaning & Preprocessing

The "content" column, containing the text of the reports, undergoes a series of cleaning, normalization, and preprocessing steps to ensure accurate and efficient analysis. These steps include:

- **String conversion**: Converting the input to a string format ensures consistency and compatibility during subsequent processing tasks.
- **Lowercase conversion**: Transforming all text to lowercase serves as a simple normalization step, reducing the complexity and variability of the input data.
- **Unicode decoding**: Removing diacritics (e.g., accented characters) and normalizing the text encoding mitigates potential discrepancies arising from different encoding formats.
- **URL and email address removal**: Eliminating URLs and email addresses reduces noise in the dataset, as these elements do not contribute valuable information for the analysis.
- **Extra whitespace removal**: Eradicating extra whitespaces improves text analysis and tokenization by ensuring that only meaningful spaces are retained.
- **Contact detail removal**: Excluding phone numbers, contact person strings, and social media references further minimizes noise in the dataset, honing the focus on relevant text.
- **Table of contents removal**: Discarding the table of contents enhances the data quality by eliminating repetitive and non-essential information.
- **Named entity removal**: Employing the spaCy model to remove human names and other named entities optimizes the text for analysis and modeling by concentrating on pertinent content.
- **Abbreviation expansion**: Utilizing the contractions library and custom functions with regular expressions, common and uncommon abbreviations are expanded to improve text interpretation.
- **Special character elimination**: Excluding all special characters, except punctuation, refines the input data. Retaining punctuation is necessary for accurate sentence tokenization and removed after sentence tokenization..
- **Tokenization and lemmatization**: Tokenizing words and sentences, and subsequently lemmatizing words using the WordNetLemmatizer from nltk, streamlines the text and reduces morphological variations.
- **Stopword removal**: Customizing the nltk stopwords list by adding or removing specific stopwords enables more precise and tailored text analysis.
- **Part-of-speech (POS) tagging**: Assigning POS tags to words and sentences enhances the text representation by providing additional linguistic information, which may be beneficial for subsequent analysis and modeling tasks.
- **Sentiment Analysis**: Basic sentiment value calculation to get first insights im terms of the sentiments in the reports within the EDA.

Spellchecking was tested with TextBlob and PySpellChecker but deliverd not useful results

In [195]:
cleaned_data = general_cleaned_data.copy(deep=True)

In [196]:
# Since the spacy model shows better results on the "raw" text, the named entity removal is conducted before all normalization and cleaning steps
spacy_model = spacy.load('en_core_web_md')
spacy_model.max_length = 1800000 # Increase max text length

def remove_named_entities(text):
    doc = spacy_model(text)
    
    named_entities = set()
    for ent in doc.ents:
        if ent.label_ in ["PERSON"]:
            named_entities.add(ent.text)
    
    named_entities_count = len(named_entities)
    
    for named_entity in named_entities:
        text = text.replace(named_entity, '')
    
    return text, named_entities_count

# Assuming cleaned_data is a pandas DataFrame with a 'content' column
cleaned_data['cleaned_content'], name_entity_count = zip(*cleaned_data['content'].apply(remove_named_entities))
print("Name entities removed:", sum(name_entity_count))

Name entities removed: 94745


In [197]:
def remove_urls(text):
    urls = re.findall(r'http\S+|www\S+|https\S+', text, flags=re.MULTILINE)
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE), len(urls)

def remove_emails(text):
    mail_addresses = re.findall(r'\S+@\S+\s?', text, flags=re.MULTILINE)
    return re.sub(r'\S+@\S+\s?', '', text, flags=re.MULTILINE), len(mail_addresses)

def remove_extra_whitespace(text):
    extra_spaces = re.findall(r'\s{2,}', text)
    return re.sub(r'\s+', ' ', text).strip(), len(extra_spaces)

cleaned_data['cleaned_content'] = cleaned_data['cleaned_content'].astype(str) # Convert all texts to string
cleaned_data['cleaned_content'] = cleaned_data['cleaned_content'].apply(lambda x: x.lower()) # Convert all texts to lower-case
cleaned_data['cleaned_content'] = cleaned_data['cleaned_content'].apply(lambda x: unidecode(x, errors="preserve")) # Remove diacritics / accented characters and unicode normalization
cleaned_data['cleaned_content'], url_count = zip(*cleaned_data['cleaned_content'].apply(remove_urls)) # Remove URLs from texts
cleaned_data['cleaned_content'], email_count = zip(*cleaned_data['cleaned_content'].apply(remove_emails)) # Remove e-mail addresses from texts
cleaned_data['cleaned_content'], extra_space_count = zip(*cleaned_data['cleaned_content'].apply(remove_extra_whitespace)) # Remove extra whitespaces from texts

print("URLs removed:", sum(url_count))
print("Mail addresses removed:", sum(email_count))
print("Extra whitespaces removed:", sum(extra_space_count))

URLs removed: 7492
Mail addresses removed: 434
Extra whitespaces removed: 149075


In [198]:
def remove_contact_details(text):
    # Remove phone numbers
    phone_regex = r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]'
    phone_count = len(re.findall(phone_regex, text))
    text = re.sub(phone_regex, '', text)

    # Remove common contact-related phrases
    contact_phrases_regex = r'\b(?:Contact Person|Phone|Tel|Fax|Mobile|E?mail|Skype|Twitter|Facebook|LinkedIn|Website):\b'
    contact_phrases_count = len(re.findall(contact_phrases_regex, text, flags=re.IGNORECASE))
    text = re.sub(contact_phrases_regex, '', text, flags=re.IGNORECASE)

    total_count = phone_count + contact_phrases_count
    return text, total_count

def remove_table_of_contents(text):
    # Remove common table of contents phrases
    toc_phrases_regex = r'\b(?:Table of Contents|Contents)\b'
    toc_phrases_count = len(re.findall(toc_phrases_regex, text, flags=re.IGNORECASE))
    text = re.sub(toc_phrases_regex, '', text, flags=re.IGNORECASE)

    # Remove content with numbering like "1. Introduction", "1.1. Background", "A. Overview", etc.
    toc_entries_regex = r'(^|\n)\s*\w+(\.\w+)*\s+\w+([\w\s]+)?'
    toc_entries_count = len(re.findall(toc_entries_regex, text))
    text = re.sub(toc_entries_regex, '', text)

    total_count = toc_phrases_count + toc_entries_count
    return text, total_count

cleaned_data['cleaned_content'], contact_count = zip(*cleaned_data['cleaned_content'].apply(remove_contact_details))
cleaned_data['cleaned_content'], toc_count = zip(*cleaned_data['cleaned_content'].apply(remove_table_of_contents))
print("Contact information removed:", sum(contact_count))
print("TOCs removed:", sum(toc_count))

Contact information removed: 44237
TOCs removed: 9603


In [199]:
def expand_contractions(text):
    expanded_text = []
    for word in text.split():
        expanded_text.append(contractions.fix(word))
    expanded_text = ' '.join(expanded_text)
    return contractions.fix(expanded_text)

cleaned_data['cleaned_content'] = cleaned_data['cleaned_content'].apply(expand_contractions)

In [200]:
# Expand custom abbreviations which are not captured by "contractions"
# Basic idea from: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
# Compile the regular expressions only once for efficiency
specific_patterns = [
    (re.compile(r"won['’]t"), "will not"),
    (re.compile(r"can['’]t"), "can not"),
]

def decontracted(phrase):
    count = 0

    # Replace specific patterns
    for pattern, replacement in specific_patterns:
        matches = len(pattern.findall(phrase))
        count += matches
        phrase = pattern.sub(replacement, phrase)

    return phrase, count

# Apply the function to expand abbreviations
cleaned_data['cleaned_content'], abbreviation_counts = zip(*cleaned_data['cleaned_content'].apply(decontracted))
print("Expanded custom abbreviations:", sum(abbreviation_counts))

Expanded custom abbreviations: 0


In [201]:
# Remove special characters excl. punctuation since this is needed by the sentence tokenization
def remove_non_alphanumeric(text, remove_punctuation=False):
    if remove_punctuation:
        pattern = r'[^a-zA-Z0-9\s]'
    else:
        pattern = r'[^a-zA-Z0-9\s.,!?\'"]'
    
    special_chars = re.findall(pattern, text)
    return re.sub(pattern, '', text), len(special_chars)

cleaned_data['cleaned_content'], special_char_count = zip(*cleaned_data['cleaned_content'].apply(remove_non_alphanumeric, remove_punctuation=False))
print("Special characters excl. punctuation removed:", sum(special_char_count))

Special characters excl. punctuation removed: 1388695


In [202]:
def tokenize_words(text):
    # Remove numbers, digits, and punctuation
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize words
    tokens = word_tokenize(text)

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens, len(tokens)

cleaned_data['word_tokens'], word_token_count = zip(*cleaned_data['cleaned_content'].apply(tokenize_words))
print("Generated word token amount:", sum(word_token_count))

Generated word token amount: 16966134


In [203]:
def tokenize_sentences(text):
    # Tokenize sentences
    tokens = sent_tokenize(text)
    
    return tokens, len(tokens)

cleaned_data['sentence_tokens'], sentence_token_count = zip(*cleaned_data['cleaned_content'].apply(tokenize_sentences))
print("Generated sentence token amount:", sum(sentence_token_count))

Generated sentence token amount: 687795


In [204]:
def remove_stopwords_from_word_tokens(tokens, custom_stopwords):
    # Remove stopwords and one-character tokens from tokens
    filtered_tokens = [
        token for token in tokens
        if token.lower() not in custom_stopwords and len(token) > 1
    ]
    
    return filtered_tokens, len(tokens) - len(filtered_tokens)

def remove_stopwords_from_sentence_tokens(sentences_list, custom_stopwords):
    filtered_sentences_list = []
    total_removed_items_count = 0

    for sentence in sentences_list:
        # Tokenize the sentence into words
        word_tokens = word_tokenize(sentence)

        # Remove stopwords, one-character tokens, digits, numbers, and special characters (excluding whitespace) from word tokens
        filtered_word_tokens = [
            re.sub(rf"[{re.escape(string.punctuation)}]", '', token) for token in word_tokens
            if token.lower() not in custom_stopwords
            and len(token) > 1
            and not re.search(r'\d', token)
            and not re.search(r'\W', token)
        ]

        # Reconstruct the sentence without the removed words and special characters
        filtered_sentence = ' '.join(filtered_word_tokens)
        removed_items_count = len(word_tokens) - len(filtered_word_tokens)
        filtered_sentences_list.append(filtered_sentence)
        total_removed_items_count += removed_items_count

    return filtered_sentences_list, total_removed_items_count

# Define custom stopwords to add or remove (the extra stopwords were identified by the TFIDF based wordcloud)
custom_stopwords = {
    'add': ['said','company','companies','year','billion','million','siemens','linde','rwe','volkswagen','symrise','porsche','sap','adidas','puma','airbus','bmw','hannover','mtu','heiderbergcement','qiagen','benz','continental','bayer','fresenius','wa', 'ha', 'eur'],
    'remove': [''] # Currently not needed
}

# Combine stopwords to filter the content of the reports
all_stopwords = set(stopwords.words('english'))
all_stopwords |= set(custom_stopwords['add'])
all_stopwords -= set(custom_stopwords['remove'])

cleaned_data['word_tokens'], stopword_count_words = zip(*cleaned_data['word_tokens'].apply(remove_stopwords_from_word_tokens, custom_stopwords=all_stopwords))
cleaned_data['sentence_tokens'], stopword_count_sentences = zip(*cleaned_data['sentence_tokens'].apply(remove_stopwords_from_sentence_tokens, custom_stopwords=all_stopwords))

print("Removed stopwords in word tokens", sum(stopword_count_words))
print("Removed stopwords in sentence tokens", sum(stopword_count_sentences))

Removed stopwords in word tokens 6817319
Removed stopwords in sentence tokens 9628193


In [211]:
# Update the cleaned content based on the cleaned word tokens
cleaned_data['cleaned_content'] = cleaned_data['word_tokens'].apply(lambda x: ' '.join(x))

In [212]:
def pos_tagging_tokens(word_tokens, sentence_list):
    # POS tagging for word tokens
    pos_tagged_word_tokens = nltk.pos_tag(word_tokens)

    # Create a dictionary to map word tokens to their POS tags, this reduces the effort to call nltk.pos_tag twice
    pos_tags_dict = dict(pos_tagged_word_tokens)

    # POS tagging for sentence tokens
    pos_tagged_sentence_list = []
    for sentence in sentence_list:
        tokenized_sentence = nltk.word_tokenize(sentence)
        pos_tagged_sentence = [(token, pos_tags_dict[token]) for token in tokenized_sentence if token in pos_tags_dict]
        pos_tagged_sentence_list.append(pos_tagged_sentence)

    return pos_tagged_word_tokens, pos_tagged_sentence_list

# Apply POS tagging
pos_tags = cleaned_data.apply(lambda row: pos_tagging_tokens(row['word_tokens'], row['sentence_tokens']), axis=1)
cleaned_data['pos_tagged_word_tokens'], cleaned_data['pos_tagged_sentence_tokens'] = zip(*pos_tags)

In [213]:
# Create checkpoint file
cleaned_data = csv_checkpoint(cleaned_data, 'cleaned_data')

Saved DataFrame to cleaned_data.csv
Loaded DataFrame from cleaned_data.csv


## Data Enrichment

Several additional information could be helpful in the further analysis, which are not included in the dataset. Therefore a small scraper is used to enrich the the dataset with the sector, industry and market capitalization of the DAX companies.

In [214]:
url = 'https://disfold.com/stock-index/dax/companies/'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table')
scraped_data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    scraped_data.append(cols)

def clean_scraped_data(data):
    cleaned_data = []
    
    for row in data:
        # Remove empty rows
        if len(row) > 0:
            # Remove the '$' and ',' signs from the market cap and convert it to float
            market_cap = float(row[3].replace('$', '').replace(',', '').replace('B', ''))
            cleaned_data.append([row[1], row[2], market_cap, row[4], row[5], row[6]])
    
    df = pd.DataFrame(cleaned_data, columns=['company_name', 'symbol', 'market_cap_in_usd_b', 'country', 'sector', 'industry'])
    
    return df

company_enrichments = clean_scraped_data(scraped_data)
company_enrichments.to_csv('../data/dax_company_sectors.csv', index=False)
company_enrichments.head()

,company_name,symbol,market_cap_in_usd_b,country,sector,industry
0,Linde plc,LIN,156.93,United Kingdom,Basic Materials,Specialty Chemicals
1,SAP SE,SAP,121.03,Germany,Technology,Software—Application
2,Siemens AG,SIE,110.13,Germany,Industrials,Specialty Industrial Machinery
3,Deutsche Telekom AG,DTE,101.78,Germany,Communication Services,Telecom Services
4,Airbus SE,AIR,96.87,Netherlands,Industrials,Aerospace & Defense


In [215]:
# Fix the ticker symbols to prevent NaN and ensure correct join conditions
company_enrichments['symbol'] = company_enrichments['symbol'].replace('SRT3', 'SRT')
company_enrichments['symbol'] = company_enrichments['symbol'].replace('HEN3', 'HNK')
company_enrichments.loc[company_enrichments['company_name'] == 'Mercedes-Benz Group AG', 'symbol'] = 'DAI'

In [216]:
cleaned_data['symbol'] = cleaned_data['symbol'].astype(pd.StringDtype())
company_enrichments['symbol'] = company_enrichments['symbol'].astype(pd.StringDtype())

# Merge the cleaned data with the enrichment
enriched_cleaned_data = pd.merge(cleaned_data, company_enrichments, how='left', on='symbol')

In [217]:
enriched_cleaned_data[enriched_cleaned_data['industry'].isnull()]

,company,content,datatype,date,domain,esg_topics,internal,symbol,title,cleaned_content,word_tokens,sentence_tokens,pos_tagged_word_tokens,pos_tagged_sentence_tokens,company_name,market_cap_in_usd_b,country,sector,industry
48,Hannover R,Sustainability Report 2020 We face up to futur...,sustainability_report,2020-03-31,NaN,"['Whistleblowing', 'Vaccine', 'Corruption', 'G...",1,HNR1,HannoverRückversicherungAG Sustainability Repo...,somewhat different approach purpose value refl...,"['somewhat', 'different', 'approach', 'purpose...","['somewhat different', 'approach', 'purpose va...","[('somewhat', 'RB'), ('different', 'JJ'), ('ap...","[[('somewhat', 'RB'), ('different', 'JJ')], [(...",NaN,NaN,NaN,NaN,NaN
76,Hannover R,Annual Report An overview Gross premium E 01 i...,annual_report,2021-03-31,NaN,"['Vaccine', 'Monopolization', 'Corruption', 'G...",1,HNR1,HannoverRückversicherungAG Annual Report 2021,group net income policyholder surplus book vue...,"['group', 'net', 'income', 'policyholder', 'su...","['group net income policyholders', 'surplus bo...","[('group', 'NN'), ('net', 'JJ'), ('income', 'N...","[[('group', 'NN'), ('net', 'JJ'), ('income', '...",NaN,NaN,NaN,NaN,NaN


Hannover R AG cannot be matched, since it is not present in the scraped data. Since there are only 2 records this is negligible and will be fixed manually.

In [218]:
enriched_cleaned_data.loc[enriched_cleaned_data['company'] == 'Hannover R', 'sector'] = 'Financials'
enriched_cleaned_data.loc[enriched_cleaned_data['company'] == 'Hannover R', 'industry'] = 'Insurance—Reinsurance'

In [219]:
# Drop redundant columns/data
enriched_cleaned_data = enriched_cleaned_data.drop(columns=['content', 'company_name', 'country'])

In [220]:
# Check final dataframe
enriched_cleaned_data.sample(10)

,company,datatype,date,domain,esg_topics,internal,symbol,title,cleaned_content,word_tokens,sentence_tokens,pos_tagged_word_tokens,pos_tagged_sentence_tokens,market_cap_in_usd_b,sector,industry
7539,Qiagen,thinktank,2021-04-27,thelancet,"['GenderDiversity', 'Vaccine', 'Social']",0,QIA,SARS-CoV-2 antibody-positivity protects agains...,sarscov2 documented raising public health conc...,"['sarscov2', 'documented', 'raising', 'public'...","['documented raising public health concerns', ...","[('sarscov2', 'NN'), ('documented', 'VBD'), ('...","[[('documented', 'VBD'), ('raising', 'VBG'), (...",11.38,Healthcare,Diagnostics & Research
7453,Puma,business,2021-12-28,marketscreener,"['HumanCapital', 'Social', 'GenderDiversity', ...",0,PUM,Puma: SOCIAL JUSTICE ADVOCATE AND MUSICIAN DEO...,want work always inspire meaningful opportunit...,"['want', 'work', 'always', 'inspire', 'meaning...","['want work always inspire meaningful', 'oppor...","[('want', 'JJ'), ('work', 'NN'), ('always', 'R...","[[('want', 'VBP'), ('work', 'NN'), ('always', ...",9.74,Consumer Discretionary,Footwear & Accessories
9901,Siemens Healthineers,general,2022-06-15,itnonline,['Social'],0,SHL,Revolutionary Technology Shortens Cardiac Scan...,imaging mechanism performance evaluation selfc...,"['imaging', 'mechanism', 'performance', 'evalu...",['imaging mechanism performance evaluation sel...,"[('imaging', 'VBG'), ('mechanism', 'NN'), ('pe...","[[('imaging', 'VBG'), ('mechanism', 'NN'), ('p...",55.11,Healthcare,Diagnostics & Research
4735,Deutsche Telekom,business,2023-02-22,business-review,"['RenewableEnergy', 'Environment', 'Social']",0,DTE,Top story of the year in telecom: 5G rollout,one industry developed rapidly last couple dec...,"['one', 'industry', 'developed', 'rapidly', 'l...",['one industries developed rapidly last couple...,"[('one', 'CD'), ('industry', 'NN'), ('develope...","[[('one', 'CD'), ('developed', 'VBD'), ('rapid...",101.78,Communication Services,Telecom Services
9166,Siemens,business,2021-04-30,marketscreener,['RenewableEnergy'],0,SIE,Siemens Gamesa Renewable Energy S A: Segundo T...,disclaimer material prepared gamesa renewable ...,"['disclaimer', 'material', 'prepared', 'gamesa...",['disclaimer material prepared gamesa renewabl...,"[('disclaimer', 'JJ'), ('material', 'NN'), ('p...","[[('disclaimer', 'NN'), ('material', 'NN'), ('...",110.13,Industrials,Specialty Industrial Machinery
9994,Siemens Healthineers,general,2021-10-03,itnonline,"['Social', 'GenderDiversity']",0,SHL,Latest Ultrasound Advances on Display at RSNA ...,december issue view chart acuson sequoia healt...,"['december', 'issue', 'view', 'chart', 'acuson...","['december issue', 'view chart', 'acuson sequo...","[('december', 'JJ'), ('issue', 'NN'), ('view',...","[[('december', 'JJ'), ('issue', 'NN')], [('vie...",55.11,Healthcare,Diagnostics & Research
10902,Zalando,general,2021-02-11,finews,"['HumanCapital', 'Recycling']",0,ZAL,CEO-Turned-Full-Time Dad Resurfaces at Swiss Bank,time caregiver three daughter back zurichbased...,"['time', 'caregiver', 'three', 'daughter', 'ba...","['time caregiver three daughters back', 'zuric...","[('time', 'NN'), ('caregiver', 'NN'), ('three'...","[[('time', 'NN'), ('caregiver', 'NN'), ('three...",10.41,Consumer Discretionary,Internet Retail
9838,Siemens Healthineers,general,2021-10-03,itnonline,"['Social', 'PatientSafety']",0,SHL,Toshiba’ s Infinix 4DCT Seamlessly Integrates ...,industry first seamless integration interventi...,"['industry', 'first', 'seamless', 'integration...",['industry first seamless integration interven...,"[('industry', 'NN'), ('first', 'RB'), ('seamle...","[[('industry', 'NN'), ('first', 'JJ'), ('seaml...",55.11,Healthcare,Diagnostics & Research
9762,Siemens Healthineers,general,2022-11-02,itnonline,['Social'],0,SHL,ASTRO 2022 Shines Spotlight on “ Cancer Breakt...,representative american society clinical oncol...,"['representative', 'american', 'society', 'cli...",['representatives american society clinical on...,"[('representat

## Calculate Sentiment Value

As a last preprocessing step, the sentiment is calculated with the (quite basic) SentimentIntensityAnalyzer.

In [223]:
sia = SentimentIntensityAnalyzer()

# Compute the sentiment of the report
def get_sentiment_score(text):
    if not isinstance(text, str):
        return 0
    sentiment = sia.polarity_scores(text)
    return sentiment['compound']

enriched_cleaned_data['sentiment_value'] = enriched_cleaned_data['cleaned_content'].apply(get_sentiment_score) # Sentence tokens +

In [224]:
# Check NaN values
enriched_cleaned_data[enriched_cleaned_data['cleaned_content'].isna()]

,company,datatype,date,domain,esg_topics,internal,symbol,title,cleaned_content,word_tokens,sentence_tokens,pos_tagged_word_tokens,pos_tagged_sentence_tokens,market_cap_in_usd_b,sector,industry,sentiment_value
1574,AkzoNobelNV,general,2021-09-09,chemistryworld,['AnimalTesting'],0,BAS,Non-animal test for skin sensitisation gets OE...,NaN,[],[],[],[],48.10,Basic Materials,Chemicals,0.0
3881,Deutsche Bank,general,2022-11-09,law360,[],0,DBK,Former Deutsche Bank Exec Says Discrimination ...,NaN,[],[''],[],[[]],24.97,Financials,Banks,0.0
3995,Deutsche Bank,business,2021-08-20,globalcapital,[],0,DBK,Deutsche Bank debuts green label in Formosa fo...,NaN,[],[''],[],[[]],24.97,Financials,Banks,0.0
7858,RWE,general,2022-10-03,energyintel,['RenewableEnergy'],0,RWE,Germany's RWE Expands US Green Power Portfolio,NaN,[],[''],[],[[]],29.97,Utilities,Utilities—Diversified,0.0


In [225]:
# Drop NaN columns
enriched_cleaned_data = enriched_cleaned_data.dropna(subset=['cleaned_content'])

In [226]:
# Create checkpoint file for further analysis
enriched_cleaned_data = csv_checkpoint(enriched_cleaned_data, 'enriched_cleaned_data')

Saved DataFrame to enriched_cleaned_data.csv
Loaded DataFrame from enriched_cleaned_data.csv


In [228]:
# Check the data after full preprocessing
enriched_cleaned_data.sample(10)

,company,datatype,date,domain,esg_topics,internal,symbol,title,cleaned_content,word_tokens,sentence_tokens,pos_tagged_word_tokens,pos_tagged_sentence_tokens,market_cap_in_usd_b,sector,industry,sentiment_value
4458,Deutsche Post,tech,2021-06-18,supplychainbrain,"['Environment', 'Social', 'Biofuel', 'LowCarbo...",0,DPW,DHL Adds Sustainable Marine Fuel Option for Fu...,air ocean freight specialist deutsche post dhl...,"['air', 'ocean', 'freight', 'specialist', 'deu...",['air ocean freight specialist deutsche post d...,"[('air', 'NN'), ('ocean', 'JJ'), ('freight', '...","[[('air', 'NN'), ('ocean', 'JJ'), ('freight', ...",45.40,Industrials,Integrated Freight & Logistics,0.9864
3216,Covestro,thinktank,2022-11-28,spglobal,['Solvents'],0,1COV,Urethane Surface Coatings - Chemical Economics...,analytics expertise sign product service cente...,"['analytics', 'expertise', 'sign', 'product', ...","['analytics expertise', 'sign product service ...","[('analytics', 'NNS'), ('expertise', 'VBP'), (...","[[('analytics', 'NNS'), ('expertise', 'VBP')],...",8.40,Basic Materials,Specialty Chemicals,0.9970
178,Adidas,general,2021-07-21,teenvogue,"['Privacy', 'GenderDiversity']",0,ADS,Olympics 2020 Athletes to Know: Rock Climber B...,teen vogue caught brightest team usa star head...,"['teen', 'vogue', 'caught', 'brightest', 'team...",['teen vogue caught brightest team usa stars h...,"[('teen', 'JJ'), ('vogue', 'NN'), ('caught', '...","[[('teen', 'JJ'), ('vogue', 'NN'), ('caught', ...",25.77,Consumer Discretionary,Footwear & Accessories,0.9993
3964,Deutsche Bank,tech,2021-07-20,finextra,"['GenderDiversity', 'HumanCapital']",0,DBK,LedgerEdge appoints Michelle Neal as CEO of US...,rolling global corporate bond trading platform...,"['rolling', 'global', 'corporate', 'bond', 'tr...",['rolling global corporate bond trading platfo...,"[('rolling', 'VBG'), ('global', 'JJ'), ('corpo...","[[('rolling', 'VBG'), ('global', 'JJ'), ('corp...",24.97,Financials,Banks,0.9840
1937,BMW,tech,2021-03-30,digitaljournal,"['HealthRisk', 'Pollution', 'Hydropower', 'Tox...",0,BMW,Indonesia scrubbing the 'world's dirtiest river ',hi looking published scabies indonesian rice f...,"['hi', 'looking', 'published', 'scabies', 'ind...","['hi looking', 'published scabies indonesian r...","[('hi', 'NN'), ('looking', 'VBG'), ('published...","[[('hi', 'NN'), ('looking', 'VBG')], [('publis...",60.24,Consumer Discretionary,Auto Manufacturers,-0.9972
580,Adidas,NaN,2021-09-28,vegconomist,"['Overconsumption', 'AnimalWelfare', 'Social']",0,ADS,Kering Ends Use of Fur at All Its Fashion Hous...,none fashion house use real animal fur include...,"['none', 'fashion', 'house', 'use', 'real', 'a...","['none fashion houses use real animal fur', 'i...","[('none', 'NN'), ('fashion', 'NN'), ('house', ...","[[('none', 'NN'), ('fashion', 'NN'), ('use', '...",25.77,Consumer Discretionary,Footwear & Accessories,0.9186
1383,AkzoNobelNV,business,2022-04-27,business-review,"['Social', 'HumanCapital']",0,BAS,BASF and its employees donate an additional € ...,much basf employee donated basf stiftung basf ...,"['much', 'basf', 'employee', 'donated', 'basf'...",['much basf employees donated basf stiftung ba...,"[('much', 'JJ'), ('basf', 'NN'), ('employee', ...","[[('much', 'JJ'), ('basf', 'NN'), ('donated', ...",48.10,Basic Materials,Chemicals,0.9950
5070,Deutsche Telekom,business,2022-08-11,marketscreener,[],0,DTE,Good news keeps on coming,july producer price rose le expected yearonyea...,"['july', 'producer', 'price', 'rose', 'le', 'e...",['us july producer prices rose less expected y...,"[('july', 'RB'), ('producer', 'NN'), ('price',...","[[('july', 'JJ'), ('producer', 'NN'), ('rose',...",101.78,Communication Services,Telecom Services,0.8957
10380,Volkswagen,general,2021-04-30,swissinfo,"['Social', 'Privacy', 'CarbonDioxide', 'LowCar...",0,VOW3,"Predicting future mobility, and remembering a ...",selfdriving car also way mature enough meet cl...,"['selfdriving', 'car', 'also', 'way', 'mature'...","['selfdriving 